In [3]:
%pip install openai

In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [8]:
import pandas as pd
data = pd.read_csv('/content/gdrive/MyDrive/amazon_review/all_files/B08HJKWBQZ - NEW AGE Naturals Advanced Hemp Gummies - Natural H 2023-12-24.csv')
data.head()

,Date,Author,Verified,Helpful,Title,Body,Rating,Images,Videos,URL,Variation,Style
0,"Reviewed in the United States on February 25, ...",Mike,yes,yes,Is this real?,I am a Registered Nurse that recently treated ...,5,-,-,https://www.amazon.com/gp/customer-reviews/R3P...,B08HJKWBQZ,Size: 90 Count (Pack of 2)
1,"Reviewed in the United States on May 11, 2021",Walter Prochorenko,yes,yes,Where's the Hemp?,"According to the label, there does not appear ...",1,-,-,https://www.amazon.com/gp/customer-reviews/R16...,B08HJKWBQZ,Size: 90 Count (Pack of 2)
2,"Reviewed in the United States on October 29, 2020",Memly,yes,yes,"Effective, Delicious, USA Made Gummies That He...",These are wonderful and I'm glad I found them....,5,-,-,https://www.amazon.com/gp/customer-reviews/R19...,B08HJKWBQZ,Size: 90 Count (Pack of 2)
3,"Reviewed in the United States on January 3, 2023",peter wells,yes,yes,Miraculous results so far.,As someone who has suffered from insomnia for ...,4,-,-,https://www.amazon.com/gp/customer-reviews/RGO...,B08HJKWBQZ,Size: 90 Count (Pack of 2)
4,"Reviewed in the United States on July 30, 2022",Skylark Poems,yes,yes,They work but.,Contrary to what one reviewer who stated that ...,4,-,-,https://www.amazon.com/gp/customer-reviews/R3J...,B08HJKWBQZ,Size: 90 Count (Pack of 2)


In [11]:
csv_path = '/content/gdrive/MyDrive/amazon_review/all_files/review.csv'
req_column = 'Body'
model_used = 'gpt-4-1106-preview'
batch_size = 12000

In [12]:
import pandas as pd
import time
from openai import OpenAI

review_df = pd.read_csv(csv_path)

API_KEY = '<YOUR API KEY>'
client = OpenAI(api_key = API_KEY)

def call_chat(system_prompt, prompt, model):

    res = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
        ]
    )
    try:
      return res.choices[0].message.content, res.usage.prompt_tokens, res.usage.completion_tokens
    except Exception as e:
      raise e

batches = []

# initialize a variable to keep track of the current batch
current_batch = ''

# loop through each review in the dataframe
for review in review_df[req_column]:
    if len(review)>batch_size:
        continue
    else:
        # if adding the current review to the current batch would exceed 1000 characters,
        # add the current batch to the list of batches and start a new batch with the current review
        if len(current_batch) + len(review) > batch_size:
            batches.append(current_batch)
            current_batch = review
        # otherwise, add the current review to the current batch
        else:
            current_batch = current_batch + '\n' +review

# add the final batch to the list of batches
batches.append(current_batch)
print("No of batches:",len(batches))

system_prompt = 'Think you are a top market researcher'
prompt_1 = '''
I will input customer reviews for a joint health supplement. Based on the following review inputs, can you give me a list of use cases ?
Give output as a string format where every use case is separated by '\n' like this: 'use_case_1, \n use_case_2, \n use_case_3'
Reviews:
{reviews}
'''

prompt_2 = '''
I will input customer reviews for a joint health supplement. Based on the following review inputs, can you give me a list of use cases and an elaborate explanation of each?
Reviews:
{reviews}
'''

prompt_3 = '''
Based on the below review, can you give me a list of things customers like and dislike about the product and an elaborate explanation of each?
Reviews:
{reviews}
'''

prompt_4 = '''
From the reviews below, quote as many good-to-read, human lingo for the below use cases, take at least 3 to 4 reviews for each use case:
Use cases:
{list_of_use_cases}

Reviews:
{reviews}
'''

result = {'Review_batch':[], 'use_cases': [], 'like_n_dislike':[],'human_lingo':[] }
start = time.time()
input_total_tokens = []
output_total_tokens = []

for batch in batches:
    use_cases, ip_tokens_1, op_tokens_1 = call_chat(system_prompt, prompt_1.format(reviews = batch ), model_used)
    output_2, ip_tokens_2, op_tokens_2 = call_chat(system_prompt, prompt_2.format(reviews = batch ), model_used)
    output_3, ip_tokens_3, op_tokens_3 = call_chat(system_prompt, prompt_3.format(reviews = batch ), model_used)
    output_4, ip_tokens_4, op_tokens_4 = call_chat(system_prompt, prompt_4.format(list_of_use_cases= use_cases,reviews = batch), model_used)

    ip_total_tokes_used = ip_tokens_1+ip_tokens_2+ip_tokens_3+ip_tokens_4
    op_total_tokes_used = op_tokens_1+op_tokens_2+op_tokens_3+op_tokens_4

    input_total_tokens.append(ip_total_tokes_used)
    output_total_tokens.append(op_total_tokes_used)

    result['Review_batch'].append(batch)
    result['use_cases'].append(output_2)
    result['like_n_dislike'].append(output_3)
    result['human_lingo'].append(output_4)

end = time.time()
print(f"time taken: {(end-start)/60} min")
print(f"Total Input Tokens: {sum(input_total_tokens)},\nTotal Output Tokens: {sum(output_total_tokens)},\nTotal cost: ${(sum(input_total_tokens)*0.01/1000)+(sum(output_total_tokens)*0.03/1000)}")
final_df = pd.DataFrame(result)


No of batches: 6
time taken: 10.990656638145447 min
Total Input Tokens: 61427,
Total Output Tokens: 17829,
Total cost: $1.14914


# Use Cases

In [13]:
for i in range(len(batches)):
  print("batch:", i+1)
  print(final_df.iloc[i]['use_cases'])

batch: 1
Based on the customer reviews provided for the joint health supplement "Flexuron," we can derive the following use cases:

1. **Use Case: Supplementing for Knee Joint Relief**
   **Elaboration**: Many consumers have reported experiencing relief from knee pain and discomfort, which has improved their ability to perform daily activities and exercise. This use case is applicable to customers seeking alternatives to pain medication or invasive treatments such as injections or surgery.

2. **Use Case: Improving Mobility and Reducing Dependence on Mobility Aids**
   **Elaboration**: Some users have found improvements in their walking abilities and reduced reliance on aids like canes after using the supplement. Therefore, it may be used to enhance overall joint mobility and quality of life for individuals with joint-related mobility restrictions.

3. **Use Case: Aid for Joint Problems Other Than Knees**
   **Elaboration**: Flexuron has been mentioned to provide relief in joints other

Likes and Dislikes

In [14]:
print(final_df.iloc[0]['like_n_dislike'])

Based on the provided reviews, here's a breakdown of what customers like and dislike about the product:

Customers like:
1. Efficacy in reducing joint pain: Multiple customers reported relief from knee, back, and hip pain, noting significant improvements in comfort and mobility.
2. Speed of results: Several reviewers were pleased with how quickly they experienced relief, with some noting improvements in as little as a few days.
3. Ease of use: Customers appreciated the convenience of taking only one pill per day, compared to other supplements that require multiple doses.
4. Alternative to surgery or medications: Some customers felt the product provided relief that could delay or avoid the need for joint replacement surgery or reduce reliance on pain medications.
5. Improvement in sleep: Reduction in pain levels led to better sleep for some individuals suffering from arthritis-related discomfort.
6. Positive secondary effects: Besides joint relief, some noted improvements in other areas

Human Lingo

In [15]:
print(final_df.iloc[0]['human_lingo'])

Relief from knee discomfort due to running:
- "Years ago... I used G/C and found relief. It probably took three or so weeks for the G/C to provide comfort for my knees which were taking a hit from running." (Review by GMcCarthy on 03/05/2019)
- "My knees are doing well--especially the right one which got most of my attention." (Review by GMcCarthy on 03/05/2019)
- "I ordered it for my knee. I had a lot of pain and it was difficult to walk if I didn’t use my cane... The pain is almost gone and sometimes, I don’t need my cane most of the time." (Review, no date provided)

Improvement in yoga flexibility and reduction of pelvic tenderness:
- "My knees were bothering me in some of my yoga stances when I had to bend the knees... I have reordered. I now have three (3) Flexuron bottles. My knees are doing well..." (Review by GMcCarthy on 03/05/2019)
- "I was pleased that a 'sore' spot on the left side of my pelvis (which was tender when I laid on my back) was greatly reduced in 'tenderness'."